<a href="https://colab.research.google.com/github/Anitayea/Linear_and_Nonlinear_Optimization/blob/recitations/recitation_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recitation 4 - Feb 16th


*   Quiz
*   Gurobi get dual variables
*   Dual problem discussion
*   Dual problem example
*   Practice problem

## Quiz


1) [T/F] Is the following statement T or F: In the simplex method, we
determine which variable leaves the basis, and then we determine which
variable should replace it.

ANSWER: FALSE - because we choose what enters first. When we decide which variable leaves, our choice depends on what variable is enters

---

For the remaining questions, consider the following tableau

\begin{array}{cccccc}
x_{1} &  & -x_{3} & 2x_{4} & +x_{5} & =1 \\ 
& x_{2} & x_{3} & +2x_{4} & -3x_{5} & =2
\end{array}

and assume that the cost to minimize is $z=2x_{2}+x_{3}$. 

---

2) [Numerical question] What is the value of the cost associated with the
basic feasible solution?


ANSWER: The basic variables are $x_1 = 1$ and $x_2 = 2$ so the rest are $0$ then plugging that in we get $z = 2x_2 + 0 = 4$

---

3) We run a simplex step in the above tableau. State if the following
statements are T/F [note that more than one statement may be true]:

(i) entering $x_{3}$ will improve the cost

(ii) entering $x_{4}$ will improve the cost

(iii) entering $x_{5}$ will improve the cost

(iv) the existing solution is optimal: entering no basic variable will
improve the cost.



ANSWER: Rewritting the objective function in terms of non-basic variables yields $z = 2(2- x_3 - 2x_4 + 3x_5) + x_3 = 4 - x_3 - 4x_4 +6x_5$

(ii) , (i)

NOTE: In optimization improving the objective depends on whether we are minimizing or maximizing. In this case we are minimizing so improving means decreasing the objective function, not increasing. 
---

4) [Multiple choice] Assume we enter $x_{4}$ in the tableau above. Which
variable should leave so that the basis remains feasible?

(i) $x_{1}$

(ii) $x_{2}$

(iii) $x_{3}$

(iv) $x_{5}$



ANSWER: $x_1$

---

5) [Numerical question] Run a pivot step where $x_{4}$ enters and $x_{1}$
leaves. What is the new cost?



ANSWER: 2 
For $x_4$ to enter subtract the first row from the second row, since $x_2$ isn't in the first equation it remains a basic variable. We get the following

\begin{array}{cccccc}
x_{1} &  & -x_{3} & 2x_{4} & +x_{5} & =1 \\ 
-x_1 & x_{2} & 2x_{3} &  & -4x_{5} & = 1
\end{array}

$z = 2x_2 + x_3 = 2$  

## Gurobi - getting dual variable

In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as grb
import numpy as np

here is an example we covered last recitation

1. (a) $c^TX$, $AX \geq b$, where $X = (x,y,z)$

c = (1,2,3), 

\begin{align}
  x+y &\geq 2\\
  -x-y &\geq -3\\
  x+z &\geq 4\\
  -x - z &\geq -5\\
  x,y,z &\geq 0
\end{align}


In [ ]:
A = np.array([[1,1,0], [-1,-1,0], [1,0,1], [-1,0,-1], [1,0,0], [0,1,0], [0,0,1]])
b = np.array([2,-3,4,-5,0,0,0])
c = np.array([1,2,3])

In [ ]:
m2 = grb.Model('p1')
x2 = m2.addMVar(shape = c.shape)
m2.setObjective(c@x2,grb.GRB.MAXIMIZE)
constrain2 = m2.addConstr(A@x2 >= b)

In [ ]:
m2.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 7 rows, 3 columns and 11 nonzeros
Model fingerprint: 0x7e24d555
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+00]
Presolve removed 7 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.100000000e+01


In [ ]:
print(m2.getAttr('x'))

[0.0, 3.0, 5.0]


In [ ]:
print(m2.getAttr('pi'))

[0.0, -2.0, 0.0, -3.0, 0.0, 0.0, 0.0]


## Dual problem discussion


In general, constrained optimization is of the form 
\begin{align}
\min f(x) \text{ s.t. } g(x) \geq b
\end{align}

$\min x$ s.t. $x\geq 1$

f(x) = x = g(x), b = 1

L(x,y) = x - y(x-1)

y + (1 - y) x

$\max y$ s.t. $y\leq 1$

[link text](https://)Constraints are often difficult to work with so we get rid of the constraints by constructing the following function 
\begin{align}
L(x,\pi) = f(x) - \pi^T(g(x) -b)\\
\pi \geq 0
\end{align}

It is constructed this way so that 
So, for feasible $x$ and $\pi \geq 0$:
\begin{align}
  &g(x) \geq b\\
  &\implies g(x) - b \geq 0\\
  \pi\geq \pi & \implies \pi^T(g(x)-b)\geq0\\
  \implies f(x) - \pi^T(g(x)-b) &\leq f(x)\\
  L(x,\pi) \leq f(x)
\end{align}

Thus we have that for feasible $x$ $\forall \pi\geq 0 \quad L(x,\pi) \leq f(x)$ so $\max_{\pi\geq0} L(x,\pi) \leq f(x)$

When constructing $L$, do so in a way that $\pi$ can send $L$ to infinity when constraints aren't statisfied for a minimization problem, and to negative infinity when the constraints aren't satisfied for a maximization

For a linear program we have $f(x) = c^Tx$ and $g(x) = Ax$ so 
\begin{align}
L(x,\pi) &= c^Tx - \pi^T(Ax-b)\\ 
&= \pi^Tb + (c - A^T\pi)^Tx
\end{align}

It is useful to view this as a game between two players, $x$ wants to minimize $L$ while $\pi$ wants to maximize it. If $x$ goes first it must set $Ax\geq b$ otherwise $\pi$ can bring $L$ to infinity. 

In this second line we've rearrange the equation and now we can see that if $\pi$ is chosen first then if $A^T\pi \leq c$ then $x$ can bring $L$ to negative infinity. Thus, if $\pi$ wants to maximize $L$ it must satisfy the constraint $A^T\pi \geq c$ and the objective is $b^T\pi$ so out dual problem is
\begin{align}
\max_{\pi\geq0}b^T\pi\\
\text{s.t. } A^T\pi \leq c
\end{align}

$x + y = 1$

is equivalent to

$x + y \geq 1$

and 

$-x - y \geq -1$

### what about equality constraints?
One way deal with equality constraints is to change them to inequality constraints. Afterall $a = b$ is equivalent to $a \leq b$ and $a\geq b$ (which can be written as $-a \leq -b$ so that the inequalities face the same direction.  

For formally, given
\begin{align}
\max c^T x\\
\text{s.t. } Ax = b\\
x\geq 0
\end{align}

can be rewritten as 
\begin{align}
\max c^T x\\
\text{s.t. } Ax \leq b\\
-Ax\leq -b\\
x\geq 0
\end{align}
(note we use $\leq$ for $\max$ this is not necessay but it is useful to be consistent)

then 
\begin{align}
L(x,y) &= c^Tx - (y_1, y_2)^T(Ax-b, -Ax+b)\\
&= (y_1, y_2)^T(b,-b) + (c - A^Ty_1 + A^Ty_2)^Tx\\
&= (y_1-y_2)^Tb + (c - A^T(y_1-y_2))^Tx\\
\end{align}

We could just set $y = (y_1,y_2)$ and have twice as many variables, but another thing we can do is set $y = y_1 -y_2$ and note that since both were positive we can view $y_1$ as the positive part of $y$ and $y_2$ as the negative part (so we could go back and replace our variables with $y_1 = y_+, y_2 = y_-$), thus we can just write 

\begin{align}
L(x,y) &= y^Tb + (c - A^Ty)^Tx
\end{align}

and have $y$ be unrestricted rather than $y\geq 0$. This yields the dual problem

\begin{align}
\min y^Tb\\
\text{s.t } A^Ty \geq c
\end{align}


---

A general procedure for finding duals is 


1.   Convert constraints to $Ax\geq b$ for minimizations and $Ax \leq b$ for maximizations
2.   Formulate $L$ and double check that objective and constraints make sense (which variables can send function to $\pm\infty$
3.   Rearrange $L$ so that $x$ only shows up once. Now the objective and constraints for the dual should be clear
      3.a If there are any equality constraints, consider combining two variables to make one unconstrained variable
4.   Formulate dual based on $L$

Note: It is useful to have two dual variables, one for equality constraints and one for inequality constraints if the constaints are of the form $A_1x=b_1$ and $A_2x\leq b_2$. It is not necessary to include positivity constraints as the latter form because positivity of $x$ is implied by the construction of $L$

### example

(a) maximize $c^TX$, $AX \geq b$, where $X = (x,y,z)$

c = (1,2,3), 

\begin{align}
  x+y &\geq 2\\
  -x-y &\geq -3\\
  x+z &\geq 4\\
  -x - z &\geq -5\\
  x,y,z &\geq 0
\end{align}


We wish to write this as 
\begin{align}
\max_{x\in R^3\\x\geq 0} c^Tx\\
\text{s.t. } Ax \leq b
\end{align}

Thus we set 
 $x = (x_1,x_2,x_3)$ 

A = [[ -1  -1  0]
 [1 1  0]
 [ -1  0  -1]
 [1  0 1]
 [ -1  0  0]
 [ 0  -1  0]
 [ 0  0  -1]]

 b = [ -2 3  -4 5  0  0  0]^T

 c = (1,2,3)^T

 Note that compared to last lecture we have swapped the signs of $A$ and $b$. This makes it so that we won't have negative signs in out future work. In general if you have a maximization problem you want your inequalities to be $\leq$ and for minimization you want $\geq$. 


So our problem is 
\begin{align}
\max_{x\in R^3} c^Tx\\
\text{s.t. } Ax \leq b
\end{align}

Let's find the dual

1st step find $L$
\begin{align}
L(x,y) &= c^Tx - y^T(Ax-b)\\
 &= -b^Ty + (c - A^Ty)^Tx
\end{align}

Note that when $Ax \leq b$ isn't satisfied $Ax - b$ is positive, thus for positive $y$ $L$ can be brought to negative infinity when the constraints aren't satisfied.

We can think of this as a two player game with one player controlling $x$ and the other $y$. One is trying to minimize $L(x,y)$ and the other is trying to maximize it. In the primal problem player $x$ goes first and wants to pick a feasible solution because otherwise player $y$ can take advantage of this.

In this example, if we take $x = (1,0,3)$ (a non-feasible point) then $(Ax-b)[0] = -1$ so sending $y[0]$ to $\infty$ sends $L(x,y)$ to $-\infty$.



Back to constructing out dual problem. We know we must minimize with respect to $y$ and that we need $c-A^Ty \leq 0$ else $x$ could send $L$ to positive infinity
\begin{align}
\min_{y\in R^7\\y\geq 0} b^Ty \\
\text{s.t. } A^Ty \geq c 
\end{align}

Now we have a minimization problem with a $\geq$ inequality constraint. 

# Practice problems
Calculate the duals of the duals we calculated above. Do you notice anything interesting? 

As a reminder we calculated the following duals:
1. The dual of
\begin{align}
\min_{x \geq 0} c^Tx\\
\text{s.t. } Ax\geq b
\end{align}
is 
\begin{align}
\max_{y \geq 0} b^Ty\\
\text{s.t. } A^Ty \leq c
\end{align}

2. The dual of 
\begin{align}
\max_{x \geq 0} c^Tx\\
\text{s.t. } Ax = b
\end{align}
is 
\begin{align}
\min_{y} b^Ty\\
\text{s.t. } A^Ty \geq c
\end{align}

